In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import IPython
from six.moves import urllib

print(tf.__version__)

2.0.0-alpha0


In [4]:
data = pd.read_csv(r'D:\MSDS-Capstone-Project\implementation\Data\12Z.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date')
data.set_index('Date', inplace=True)
del data['lifted index (500-100mb)']
data.head()

,2mTemp,2mDpt,sfcCAPE,sfcCIN,sfcPotentialTmp,sfcPressure,PWAT,2mRH,srh,lifted index 180-0 mb above gnd Best (4-layer),...,500mb wind speed,700mb Geopotenial height,700mb Temperature,700mb wind speed,850mb Geopotenial height,850mb Temperature,850mb wind speed,925mb Geopotenial height,925mb Temperature,925mb wind speed
Date,,,,,,,,,,,,,,,,,,,,,
1979-01-01,264.188,261.349,71.25900,-2.811410,268.290,94292.3,10.59400,80.7518,120.5190,11.7502,...,29.8053,2988.24,260.828,16.8339,1490.52,265.006,10.50520,833.208,265.451,9.06277
1979-01-02,258.685,254.674,17.17490,-0.669441,262.562,94780.1,4.03745,73.9237,78.2061,16.7454,...,30.9239,2969.31,256.752,18.9068,1503.93,258.143,13.28200,863.205,260.174,11.21120
1979-01-03,261.706,257.618,7.58657,0.103668,265.158,94725.0,5.43055,74.0297,113.5680,15.9920,...,27.1962,3016.15,262.246,15.8660,1514.07,264.869,9.47904,859.342,265.686,7.66035
1979-01-04,264.554,261.212,8.17602,-0.332742,267.677,95013.9,8.41552,78.6522,178.0570,12.6097,...,27.0118,3050.98,263.788,14.4288,1542.76,265.632,9.49387,885.554,267.294,7.90352
1979-01-05,264.927,262.247,17.40500,-0.761432,267.827,94902.7,9.82837,82.7518,172.2430,10.3466,...,23.7734,3059.86,264.751,12.8224,1541.93,267.945,8.39947,880.134,268.474,7.50598


In [5]:
tor_count = pd.read_csv(r'D:\MSDS-Capstone-Project\implementation\Data\tornadoCounts.csv', index_col=1)
tor_count = tor_count.loc[:, ~tor_count.columns.str.contains('^Unnamed')]
tor_count.index = pd.to_datetime(tor_count.index)
tor_count.tail(20)

,count
date,
2013-12-12,0
2013-12-13,0
2013-12-14,0
2013-12-15,0
2013-12-16,0
2013-12-17,0
2013-12-18,0
2013-12-19,0
2013-12-20,3


In [6]:
tor_count.rename(columns={'count':'COUNT'}, inplace=True)
tor_count.loc[tor_count.COUNT < 5, 'countBinary'] = 0.
tor_count.loc[tor_count.COUNT >= 5, 'countBinary'] = 1.
tor_count.countBinary = tor_count.countBinary.astype(int)
tor_count.tail(20)

,COUNT,countBinary
date,,
2013-12-12,0,0
2013-12-13,0,0
2013-12-14,0,0
2013-12-15,0,0
2013-12-16,0,0
2013-12-17,0,0
2013-12-18,0,0
2013-12-19,0,0
2013-12-20,3,0


In [7]:
merge = pd.merge(data,tor_count, how='inner', left_index=True, right_index=True)
merge = merge.sample(frac=1)
merge.tail(20)

,2mTemp,2mDpt,sfcCAPE,sfcCIN,sfcPotentialTmp,sfcPressure,PWAT,2mRH,srh,lifted index 180-0 mb above gnd Best (4-layer),...,700mb Temperature,700mb wind speed,850mb Geopotenial height,850mb Temperature,850mb wind speed,925mb Geopotenial height,925mb Temperature,925mb wind speed,COUNT,countBinary
Date,,,,,,,,,,,,,,,,,,,,,
1993-11-09,278.260,273.687,5.778040,-3.857850,281.035,94406.7,15.08410,74.5077,94.4163,6.114490,...,269.196,10.02350,1542.38,278.370,5.42181,847.884,281.675,3.93262,0,0
1991-05-26,291.056,286.883,432.156000,-70.937200,296.168,93205.5,27.00150,79.7470,75.1864,-0.245066,...,280.254,7.32758,1472.31,289.598,7.04994,747.756,293.164,6.03643,12,1
1993-03-19,277.353,274.473,22.102600,-25.181500,281.330,93888.1,14.68130,83.7400,177.2880,4.843910,...,272.078,10.82610,1491.37,279.186,9.99173,798.303,280.248,10.16570,0,0
2007-09-19,290.095,286.622,557.503000,-40.014400,293.737,93967.0,26.77000,81.6729,72.6563,1.156700,...,280.527,7.32292,1540.63,288.812,6.25625,818.329,292.336,6.58514,0,0
2010-02-25,268.388,263.011,1.507050,0.131166,272.443,93706.5,5.98789,72.0326,104.1700,10.778300,...,267.531,11.31060,1451.63,273.620,7.30477,775.146,273.902,6.90379,0,0
1984-11-18,277.015,273.908,57.805300,-8.199870,281.445,93632.7,15.64980,81.6483,103.9170,6.900880,...,270.078,12.03070,1470.89,276.689,8.28278,782.045,278.424,7.67745,0,0
2007-08-30,291.821,287.978,595.358000,-57.361200,295.392,94301.9,32.18340,80.0326,52.6098,0.414493,...,281.352,5.58305,1575.24,289.979,4.67310,847.956,293.991,5.01518,0,0
2005-02-13,279.400,276.916,22.551300,-13.987100,284.213,92997.6,18.28260,85.6160,162.4480,4.671880,...,272.237,14.01300,1427.21,281.258,13.24730,726.183,282.825,11.94730,0,0
2010-11-17,277.244,272.221,0.672374,-1.868160,281.173,93734.4,9.06170,73.7727,104.2310,7.872010,...,268.752,11.86650,1482.48,277.321,9.05365,790.373,280.459,9.10251,0,0


In [8]:
merge.head()

,2mTemp,2mDpt,sfcCAPE,sfcCIN,sfcPotentialTmp,sfcPressure,PWAT,2mRH,srh,lifted index 180-0 mb above gnd Best (4-layer),...,700mb Temperature,700mb wind speed,850mb Geopotenial height,850mb Temperature,850mb wind speed,925mb Geopotenial height,925mb Temperature,925mb wind speed,COUNT,countBinary
Date,,,,,,,,,,,,,,,,,,,,,
1989-07-04,294.778,288.841,764.6680,-123.25600,299.111,93863.1,28.21450,72.3841,69.9062,-1.86218,...,283.216,7.64752,1541.60,293.106,6.60583,807.820,297.116,5.46926,0,0
2007-01-07,275.278,272.515,34.6699,-4.71256,278.729,93866.8,12.88890,82.6568,99.1625,7.20784,...,266.668,15.82940,1495.21,274.965,13.01880,808.731,278.114,10.35490,0,0
2002-01-17,274.255,269.662,11.4488,-2.07099,277.916,93871.6,11.51240,73.9697,120.5780,9.31291,...,267.541,15.17180,1482.25,273.030,8.67548,802.500,275.944,6.59866,0,0
1984-09-20,288.468,281.627,15.7453,-9.22618,291.354,93615.5,20.58340,66.2377,66.2141,4.14231,...,280.052,6.86704,1502.65,288.796,5.39810,781.869,291.944,5.96519,0,0
1981-12-26,272.211,268.279,19.2212,-3.96528,276.253,93086.7,8.67172,77.4376,122.0640,6.08400,...,265.167,14.65920,1417.15,274.615,9.04004,733.306,276.811,7.99730,0,0


In [9]:
merge.to_csv(r'D:\MSDS-Capstone-Project\implementation\Data\12Z-randomized.csv')